### Load Modules

In [1]:
import sys
from pathlib import Path

sys.path.insert(1, str(Path.cwd().parent))

In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm

import gensim
from gensim.models import FastText
from gensim.parsing.porter import PorterStemmer
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tld.config import word_embedding_dir

### Helper Functions

In [3]:
# Help function to determine best size
def calc_max_seq_len(data_df):
    titles = data_df['title']
    title_lengths = (titles.str.count(' ')+1).fillna(0).astype(np.int)
    max_title_len = title_lengths.quantile(0.95, interpolation='higher')

    descriptions = data_df['description']
    desc_lengths = (descriptions.str.count(' ')+1).fillna(0).astype(np.int)
    max_desc_len = desc_lengths.quantile(0.95, interpolation='higher')
    
    return max_title_len, max_desc_len

In [4]:
def load_issues(s):
    filename = '../data/processed/issues_'+s.lower()+'.csv'
    issues = pd.read_csv(filename, encoding="UTF-8", low_memory=False, index_col=['issue_id'], sep=";")
    print("Number of Issues: " + str(len(issues)))
    
    return issues

### Parameters & Constants

In [5]:
tqdm.pandas()
p = PorterStemmer()

stopWords = set(stopwords.words('english'))
stopWords = stopWords.union(set(['com', 'org', 'java']))

EMBEDDING_DIM = 20
MAX_SEQUENCE_LENGTH = 300

In [6]:
SOURCES = ['Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 
           'JiraEcosystem', 'MariaDB', 'Mindville', 'Mojang', 'MongoDB', 
           'Qt', 'RedHat', 'Sakai', 'SecondLife', 'Sonatype', 'Spring', 'Apache']

### Word2Vec

In [13]:
def train_w2v(s):
    print(s.upper())

    output_dir = word_embedding_dir / 'w2v'
    output_dir.mkdir(parents=True, exist_ok=True)
    
    issues = load_issues(s)
    
    issues['title'].fillna(' ', inplace=True)
    issues['description'].fillna(' ', inplace=True) 
    
    max_title_len, max_desc_len = calc_max_seq_len(issues)
    
    MAX_SEQUENCE_LENGTH = max_title_len + max_desc_len
    
    textual_data = []
    for issue_id in tqdm(issues.index):
        temp = []
        for word in word_tokenize(str(issues['title'].loc[issue_id])):
            word = word.lower()
            if word not in stopWords:
                word = p.stem(word)
                temp.append(word)
        textual_data.append(temp)
        temp = []
        for word in word_tokenize(str(issues['description'].loc[issue_id])):
            word = word.lower()
            if word not in stopWords:
                word = p.stem(word)
                temp.append(word)                
        textual_data.append(temp)
        
    model_CBOW = gensim.models.Word2Vec(textual_data, vector_size=EMBEDDING_DIM, window=5, min_count=2, workers=4, sg=0)
    model_CBOW.save(str(output_dir / f'{s.lower()}W2V.model'))
    
    text = []
    word_count = 0

    for issue_id in tqdm(issues.index):
        temp = []
        for word in word_tokenize(str(issues['title'].loc[issue_id])):
            word = word.lower()
            if word not in stopWords:
                word = p.stem(word)
                try:
                    word_index = model_CBOW.wv.key_to_index[word]
                    temp.append(word_index)
                except:
                    word_count += 1
        temp = temp[:max_title_len-1]
        for word in word_tokenize(str(issues['description'].loc[issue_id])):
            word = word.lower()
            if word not in stopWords:
                word = p.stem(word)
                try:
                    word_index = model_CBOW.wv.key_to_index[word]
                    temp.append(word_index)
                except:
                    word_count += 1
        text.append(temp)
    print(word_count)
    
    text_data = pad_sequences(text, maxlen=MAX_SEQUENCE_LENGTH)
    
    print(text_data[0].shape)

    np.save(file=output_dir / f'text_data_{s.lower()}.npy', arr=text_data)

In [14]:
for s in ['HYPERLEDGER']:
    train_w2v(s)
    print("--------------")

HYPERLEDGER
Number of Issues: 28146


C:\Users\Tim\AppData\Local\Temp\ipykernel_13172\1211113016.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  title_lengths = (titles.str.count(' ')+1).fillna(0).astype(np.int)
C:\Users\Tim\AppData\Local\Temp\ipykernel_13172\1211113016.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current 

75499
(282,)
--------------


### FASTTEXT

In [12]:
def train_fasttext(s):
    print(s.upper())

    output_dir = word_embedding_dir / 'fasttext'
    output_dir.mkdir(parents=True, exist_ok=True)
    
    issues = load_issues(s)
    
    issues['title'].fillna(' ', inplace=True)
    issues['description'].fillna(' ', inplace=True) 
      
    max_title_len, max_desc_len = calc_max_seq_len(issues)
    
    MAX_SEQUENCE_LENGTH = max_title_len + max_desc_len
    
    textual_data = []
    for issue_id in tqdm(issues.index):
        temp = []
        for word in word_tokenize(str(issues['title'].loc[issue_id])):
            word = word.lower()
            if word not in stopWords:
                word = p.stem(word)
                temp.append(word)
        textual_data.append(temp)
        temp = []
        for word in word_tokenize(str(issues['description'].loc[issue_id])):
            word = word.lower()
            if word not in stopWords:
                word = p.stem(word)
                temp.append(word)                
        textual_data.append(temp)
        
    model_CBOW = FastText(textual_data, vector_size=EMBEDDING_DIM, window=5, min_count=2, workers=4, sg=0)
    model_CBOW.save(str(output_dir / f'{s}fasttext.model'))
    
    text = []
    word_count = 0

    for issue_id in tqdm(issues.index):
        temp = []
        for word in word_tokenize(str(issues['title'].loc[issue_id])):
            word = word.lower()
            if word not in stopWords:
                word = p.stem(word)
                try:
                    word_index = model_CBOW.wv.key_to_index[word]
                    temp.append(word_index)
                except:
                    word_count += 1
        for word in word_tokenize(str(issues['description'].loc[issue_id])):
            word = word.lower()
            if word not in stopWords:
                word = p.stem(word)
                try:
                    word_index = model_CBOW.wv.key_to_index[word]
                    temp.append(word_index)
                except:
                    word_count += 1
        text.append(temp)
    print(word_count)
    
    text_data = pad_sequences(text, maxlen=MAX_SEQUENCE_LENGTH)
    
    print(text_data[0].shape)
    
    np.save(file=output_dir / f'text_data_{s}.npy', arr=text_data)

In [13]:
for s in SOURCES:
    train_fasttext(s)
    print("--------------")

JIRA


  0%|                                                                                                                                                                               | 9/265343 [00:00<49:20, 89.63it/s]

Number of Issues: 265343
Feature Size: 9


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 265343/265343 [37:24<00:00, 118.24it/s]


483358
--------------
MONGODB


...

5079
--------------
REDHAT


  0%|                                                                                                                                                                             | 89/315797 [00:00<06:04, 866.61it/s]

Number of Issues: 315797
Feature Size: 9


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 315797/315797 [09:16<00:00, 567.84it/s]


829562
--------------
